In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from tqdm import tqdm
import sys
sys.path.append('fbsource/fbcode/surreal/')

kitti_root = Path("data/kitti")  # where to install the data

# Download the image lists

In [ ]:
file_lists = ["test1_files.txt", "test2_files.txt", "train_files.txt"]
list_urls = [
    "https://raw.githubusercontent.com/shiyujiao/HighlyAccurate/main/dataLoader/{x}"
    for x in file_lists
]
for url in list_urls:
    !curl $(fwdproxy-config curl) $url --output $kitti_root/{Path(url).name}
!wc -l {kitti_root}/test2_files.txt
!wc -l {kitti_root}/train_files.txt

In [ ]:
seqs = {}
for x in file_lists:
    with open(kitti_root / x, "r") as fid:
        info = fid.read()
    seqs |= {line.split()[0].split("/")[1][:-len("_sync")] for line in info.split("\n") if line}
dates = {"_".join(s.split("_")[:3]) for s in seqs}
print(len(seqs), len(dates))

for seq in tqdm(seqs):
    print("working on", seq)
    date = "_".join(seq.split("_")[:3])
    url = f"https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/{seq}/{seq}_sync.zip"
    out_path = kitti_root / Path(url).name
    if (kitti_root / (seq+"_sync")).exists():
        continue
    !curl $(fwdproxy-config curl) $url --output $out_path
    !unzip -q $out_path -d $kitti_root
    dir_ = kitti_root / date / f"{seq}_sync"
    !rm -rf $dir_/image_0[0,1,3] $dir_/velodyne_points
    out_path.unlink()

for date in tqdm(dates):
    print(date)
    url = f"https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/{date}_calib.zip"
    out_path = kitti_root / Path(url).name
    !curl -s $(fwdproxy-config curl) $url --output $out_path
    !unzip -q -o $out_path -d $kitti_root
    out_path.unlink()

In [ ]:
with open(kitti_root / "test2_files.txt", "r") as fid:
    info = fid.read()
seqs = {line.split()[0].split("/")[1][:-len("_sync")] for line in info.split("\n") if line}
dates = {"_".join(s.split("_")[:3]) for s in seqs}

from tqdm import tqdm
for seq in tqdm(seqs):
    print(seq)
    url = f"https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/{seq}/{seq}_sync.zip"
    out_path = kitti_root / Path(url).name
    if (kitti_root / (seq+"_sync")).exists():
        continue
    !curl $(fwdproxy-config curl) $url --output $out_path
    !unzip -q $out_path -d $kitti_root
    out_path.unlink()
for date in tqdm(dates):
    print(date)
    url = f"https://s3.eu-central-1.amazonaws.com/avg-kitti/raw_data/{date}_calib.zip"
    out_path = kitti_root / Path(url).name
    !curl -s $(fwdproxy-config curl) $url --output $out_path
    !unzip -q -o $out_path -d $kitti_root
    out_path.unlink()

In [ ]:
from maploc.data.kitti import prepare_osm
prepare_osm(kitti_root, seqs)

In [ ]:
archive = kitti_root.parent / (kitti_root.name +".tar.gz")
!tar -czf -C {kitti_root.parent} {archive} {kitti_root}
!manifold put --overwrite {archive} bucket/tree/maploc/data/{archive.name}